In [1]:
import os
import json
import numpy as np
import pandas as pd
import requests
import time
import threading
import re
import codecs
from multiprocessing.dummy import Pool as ThreadPool


In [2]:
#настройки показа в панде
pd.options.display.max_columns = 100 #def100
pd.options.display.max_colwidth = 100 #def100
pd.options.display.max_rows = 200 #def 100

In [3]:
# время работы скрипта
# лимит обращений отсутствует
# время - каждые через ровные 10 минут, чтобы ровно по сетке, начиная с 00:00

# get streams' summary
# https://api.twitch.tv/kraken/streams/summary

# результат - статья на хабре, данные для стримеров

# № users, that prefer you on, and prefer other on you

# размер получаемых данных можно уменьшить, убрав ненужное

# next step - look data on followers

# parallel requests

# working on and with missing data


# содержание, оглавление и закладки для jupiter, чтобы можно было работать с большими файлами

# что за двойное ";;" при сохранение пандой в csv?
# меняет ли excel кодировку? 
# пока загружается не все при чем по неизвестным причинам (попробовать сохранить с другим разделителем?!)


# todo:
# 1) проверить, что все файлы сохранились в кодировке utf-8
# 2) сделать скрипт загрузки в mysql




# на что может влиять стример - расписание, игра, кооперация с другими стримерами
# таргетом могут быть не раскрученые канала, а раскручивающиеся с аналитикой кто от них, когда и почему ушел
# но тогда нужно собирать больше данных

# connector only for python 3.4



# собрать большое количество данных на специально подобранную инфраструктуру,
# это уже объективный барьер и потому конкурентное преимущество
# иметь данные - это потенциал




#загрузка csv в Mysql

# опция -forced почему-то работает не всегда, точнее для тех ошибок,
# которые были встречены на данный момент, она вообще не работает

# название игры, включающее ";" не проблема для загрузки csv в mysql. хотя отоброжается как несколько полей в xlsx

# иногда число просмотров канало отражается как "", вместо "0", тогда импорт csv в mysql не происходил из-за ошибки
# нужно менять "" на "0"

# есть некоторый дубляж при загрузке данных, когда один и тотже стрим попадает на разные оффсеты
# например, в 2015_12_17___18_00.csv,
# стрим от oushu_genji попал в 2 офсета c 666 и 679 зрителей соответственно:
# https://api.twitch.tv/kraken/streams?limit=100&offset=100
# https://api.twitch.tv/kraken/streams?limit=100&offset=0

# ошибка, если "\" встречается в названии игры или где-то еще из-за опции "enclosed by "\""
# при импорте через wizzard все нормально загружается
# нужна ли вообще опции "enclosed by?"
# нет, нельзя, т.к. запятые будут лишние, например в "Warhammer 40,000: Dawn of War II - Retribution"
# можно ли сделать, чтобы mysql при загрузке не воспринимал "\" как особый символ?
# Да, нужно дописать в команду "ESCAPED BY ''" после "enclosed by ..."

# название игры, вроде может отсутствовать, и это будет ок для импорта


# долгие поля mysql не воспринимает
# например, для такого названия, содержащего зачем-то еще и лишнюю инфу
# "The Bureau: XCOM Declassified (32-bit, DX9) - D:\Steam\steamapps\common\The Bureau\binaries\win32\"



# also it would be good to gather all games summary from twitch

# 



# погрешность из-за того, что некоторые каналы дублируюца в одном дампе (2% # просмотров для одного дампа)


In [24]:
way = 'G:\!datascience\twitch\\'
way1 = r'G:\\!datascience\twitch\\'
way2 = r'G:\\!datascience\twitch\parsed\\'
way3 = r'G:\\!datascience\twitch\reserved\\'
way4 = r'G:\\!datascience\twitch\load\\'
way10 =  'G:\!datascience\\twitch\\analytics\games_split1.csv'
way11 = 'G:\!datascience\\twitch\\analytics\\'

## 1 Get stream data

In [20]:
# twitch requires headers in requests

global headers
headers = {'client_id': 'oub6145j9kw161j9wnq6ayrcfnti3hw',
           'Accept': 'application/vnd.twitchtv.v3+json',
           'content-type': 'application/json',
          }


In [6]:
def get_chat_data(chan_n):
    adr_chat = 'https://tmi.twitch.tv/group/user/'
    ch_ct_add = {}
    ch_ct_add['chatters'] = []
    ch_ct_add['status'] = ''  
    ch_ct_add['try_number'] = 0
    nt = 3 # number of tries
    gct_sleep = 0.1 # sleep time if error and try again
    while nt > 0:
        nt+=-1
        ch_ct_add['try_number']+= 1      
        req = requests.get(adr_chat + chan_n + '/chatters')
        if req.status_code == 200:
            ch_ct_add['chatters'] = req.json()
            ch_ct_add['status'] = 'collected'    
            ch_ct_add['channel'] = chan_n
            return(ch_ct_add)
        else:
            time.sleep(gct_sleep)
            ch_ct_add['status'] = 'error'
    return(ch_ct_add)

In [7]:
def get_stream_list(adr):
    max_nt_stl = 3 # number of tries to get data
    curr_nt_stl = 0
    stl_get_status = 0
    while (stl_get_status == 0) and (curr_nt_stl < max_nt_stl):
        req =  requests.get(adr, headers)
        curr_nt_stl+= 1
        if req.status_code == 200:
            stl_get_status = 1 # received
            stl_raw = req.json()
        else:
            stl_get_status = 0 # failed to receive
            stl_raw = []
            time.sleep(stL_raw_sleep_time)
    data = {}
    data['adr'] = adr
    data['status'] = stl_get_status
    data['number_tries'] = curr_nt_stl
    data['streams'] = stl_raw
    return(data)

In [8]:
time_start = time.ctime()
time_start_short = str(time.strftime('%Y_%m_%d___%H_%M'))

In [24]:
# get the summary and the list of all active streams (channels)

adr_summary = 'https://api.twitch.tv/kraken/streams/summary'
tw_summary = requests.get(adr_summary, headers).json()
ttlvn = tw_summary['viewers'] # total viewers now  
ttlv_pt = 0 # progressive total for viewers count
stL_raw_sleep_time = 1 # time between tries to get stl_raw data in case of connection error
stl_get_const = 3 # assumptionL: no more than 3*100 = 300 can be started during the gata gathering

adr_list = ['https://api.twitch.tv/kraken/streams?limit=100&offset=' + str(i*100) for i in range(int(tw_summary['channels']/100) + stl_get_const)]

pool = ThreadPool(100) 
streams = pool.map(get_stream_list, adr_list)
pool.close() 
pool.join() 


In [25]:
# get the list of top most viewed channels having 90% of viewers in total

viewers_cut_const = 0.9 # get channel and chat data only for top channels having 90% of all twitch viewers in total atm
k = 0 
ttlv_pt = 0
gcsl = [] # get chat stream list
for i in range(len(streams)):
    for j in range(len(streams[i]['streams']['streams'])):
        k+=1
        ttlv_pt+= streams[i]['streams']['streams'][j]['viewers']
        if ttlv_pt/ttlvn <= viewers_cut_const:
            gcsl.append(streams[i]['streams']['streams'][j]['channel']['name'])

In [26]:
# get chat data for 90% viewers

pool = ThreadPool(100) 
ch_ct = pool.map(get_chat_data, gcsl)
pool.close() 
pool.join() 


In [27]:
# count errors

chat_error = 0
for i in range(len(ch_ct)):
    if ch_ct[i]['status'] != 'collected':
        chat_error+=1

stream_batch_error = 0
for i in range(len(streams)):
    if streams[i]['status'] != 1:
        stream_batch_error+=1
        

In [28]:
# structuring the resulting file in a json format
time_end = time.ctime()
dump_file ={}
log = {}
time_info = {}
time_info['time_start'] = time_start
time_info['time_end'] = time_end
log['time']=time
error_count = {}
error_count['stream_batch'] = stream_batch_error
error_count['chat'] = chat_error
log['error_count'] = error_count

dump_file['summary'] = tw_summary
dump_file['streams'] = streams
dump_file['ch_ct'] = ch_ct
dump_file['log'] = time_info

In [29]:
# final writing of gathred data
with open(time_start_short +str('__v3'), 'w') as outfile:
    json.dump(dump_file, outfile)

In [ ]:
import sys
print (sys.version)

In [3]:
from pymongo import MongoClient
import gridfs

In [4]:
client = MongoClient()

In [11]:
# # tutorial stuff
# db = client['primer']
# coll = db['dataset']

In [23]:
db = client['Twitch1']
collection = db['Twitch1_import1']

In [32]:
import re

In [26]:
# get list of files in a directory
# script
dirp = os.listdir(way1)

In [80]:
# script
# read a json file

f1 =[]

with open(way1+new_files[0]) as f:
    for line in f:
        f1.append(json.loads(line))

In [31]:
dirp[3300]

'2016_01_15___21_50__v3'

In [75]:
new_files = re.findall('\w*__v3', '|'.join(dirp))


In [77]:
new_files[0]

'2016_01_05___23_00__v3'

In [90]:
collection.insert_one(f1[0]['summary']).inserted_id

ObjectId('57f94fcf583a652494c2c9e3')

In [91]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Twitch1'), 'Twitch1_import1')

In [89]:
f1[0]['summary']

{'_links': {'self': 'https://api.twitch.tv/kraken/streams/summary'},
 'channels': 24780,
 'viewers': 945794}

In [95]:
collection.find_one()

{'_id': ObjectId('57f94fcf583a652494c2c9e3'),
 '_links': {'self': 'https://api.twitch.tv/kraken/streams/summary'},
 'channels': 24780,
 'viewers': 945794}

In [101]:
dic = {}
dic['1']=f1[0]

In [102]:
collection.insert_one(dic).inserted_id

DocumentTooLarge: BSON document too large (58755513 bytes) - the connected server supports BSON document sizes up to 16793598 bytes.

In [104]:
fs = gridfs.GridFS(db)

In [115]:
a = fs.put(f1[0], encoding = 'utf-8' )

TypeError: can only write strings or file-like objects

In [112]:
fs.get(a).read()

b'G:\\\\!datascience\\twitch\\\\2016_01_05___23_00__v3'

In [107]:
help(fs.put)

Help on method put in module gridfs:

put(data, **kwargs) method of gridfs.GridFS instance
    Put data in GridFS as a new file.
    
    Equivalent to doing::
    
      try:
          f = new_file(**kwargs)
          f.write(data)
      finally:
          f.close()
    
    `data` can be either an instance of :class:`str` (:class:`bytes`
    in python 3) or a file-like object providing a :meth:`read` method.
    If an `encoding` keyword argument is passed, `data` can also be a
    :class:`unicode` (:class:`str` in python 3) instance, which will
    be encoded as `encoding` before being written. Any keyword arguments
    will be passed through to the created file - see
    :meth:`~gridfs.grid_file.GridIn` for possible arguments. Returns the
    ``"_id"`` of the created file.
    
    If the ``"_id"`` of the file is manually specified, it must
    not already exist in GridFS. Otherwise
    :class:`~gridfs.errors.FileExists` is raised.
    
    :Parameters:
      - `data`: data to be wr